# This notebook includes a simple demo for flood risk assessment

## It uses flood hazard water depth scenario RP100 and population map for 2018

In [2]:
# Added to requirements.txt, but need to specify version? Then remove here
#!pip -q install geopandas
#!apt -q install libspatialindex-dev
#!pip -q install osmnx
#!pip -q install rasterio
#!pip -q install rasterstats
#!pip -q install geojson

### Import required libraries

In [3]:
# Required libraries
import geopandas as gpd
import pandas as pd
import rasterio
from rasterio.plot import show
from rasterstats import zonal_stats, utils
import rasterio.mask
import gdal, gdalconst
from osgeo import ogr, osr
from shapely import wkb
from matplotlib import pyplot
from shapely.geometry import shape

ModuleNotFoundError: No module named 'gdal'

In [ ]:
# All prev bunch (not all required)

import seaborn as sns
import geojson
import json
import geopandas as gpd
import pandas as pd
from shapely.geometry import shape, box, mapping, Point, Polygon
import numpy as np
import osmnx as ox
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_squared_error, mean_absolute_error
import math
import rasterio
from rasterio.plot import show
from rasterstats import zonal_stats, utils
import gdal, gdalconst
import ee
from osgeo import ogr, osr
from shapely import wkb
import rasterio.mask
from matplotlib import pyplot
from shapely.geometry import shape

Do we need this class stuff?

In [ ]:
#class to work with tiff datasets
class Raster:
    
    def __init__(self, filename):
        self.filename = filename
    #get data using bounding box
    def dataFromBounds(self, band, bounds):
        with rasterio.open(self.filename) as src:
            self.noData = src.nodatavals[0]
            window = rasterio.windows.from_bounds(bounds[0],bounds[1],bounds[2],bounds[3], src.transform)
            self.boundsData = src.read(band, window=window)
            self.boundsTransform = src.window_transform(window)
    #get data using polygon for masking
    def dataFromShape(self,band,shape):
        with rasterio.open(self.filename) as src:
            self.noData = src.nodatavals[0]
            self.maskData, self.maskTransform = rasterio.mask.mask(src, [shape], crop=True, nodata=0)
            self.maskData = self.maskData[band-1, :, :]


### Load input data: flood hazard (RP100) and population (Landscan 2018)

All data already share the same crs (epsg 4326)

In [ ]:
#fd represents the flood raster
fd = Raster('/data/ssudan/hzd-ss-fl-rp100.tif')
# or simply use rasterio open directly and delete class stuff?
fd = rasterio.open('/data/ssudan/hzd-ss-fl-rp100.tif')

#pop represents population data
pop = Raster('/data/ssudan/exp-ss-pop-landscan18.tif')

#adm2 represents administrative units
adm0 = ogr.Open('/data/ssudan/SS_ADM0.shp')
adm1 = ogr.Open('/data/ssudan/SS_ADM1.shp')
adm2 = ogr.Open('/data/ssudan/SS_ADM2.shp')

#print attributes in shapefile to help grab what's needed for zonal stats later
print([defn.GetFieldDefn(i).GetName() for i in range(defn.GetFieldCount())])

## 1. Flooded area

In [ ]:
target = target.to_crs(flood_crs)
flood_dict = {
    0:'non-flooded',
    1:"flooded"
}
zs = pd.DataFrame(zonal_stats(target, flood_data, affine=meta['transform'], categorical=True, category_map=flood_dict))

area = target[['ADM2_EN','ADM1_EN_1','Area_km2','Area2']].join(zs)

area.loc[:,"cells"] = area['non-flooded']+area['flooded']

area.loc[:,'Area_Cells'] = area['cells']*0.0004

area.loc[:,'Area_Flooded'] = area['flooded']*0.0004

area.loc[:,'Area_Flooded_pct'] = area['Area_Flooded']/area['Area_Cells']

area.head()

### Export data

In [ ]:
area.to_csv(os.path.join(out_folder, "area_flooded.csv"))

In [ ]:
out_folder = os.path.join(data_dir, "output")
if not os.path.exists(out_folder):
    os.mkdir(out_folder)

## 2. Flooded population

### Create flood hazard extent mask using a threshold of 15 cm

In [ ]:
# turn values > 0.15 (threshold) into boolean mask
fd_mask = bool(fd > 0.15 )

### Estimate population within hazard extent and plot as map

In [ ]:
pop_fd = fd_max * pop

In [ ]:
#print total population affected
totalPop = 0
for el in zonalStatsFB:
  if not el['sum'] is None:
    totalPop = totalPop + el['sum']

print(totalPop)

In [ ]:
plot pop_fd over osm basemap

### Summary for ADM2 level - zonal statistics

In [ ]:
# total population affected
totalPop = 0
for el in zonalStatsFB:
  if not el['sum'] is None:
    totalPop = totalPop + el['sum']

print(totalPop)

In [ ]:
# population affected for each ADM2 unit
target = target.to_crs(pop_crs)
zs_landscan = pd.DataFrame(zonal_stats(target, pop_fd, affine=landscan.meta['transform'], stats='sum')).rename(columns = {'sum':'pop_flooded_rp100'})

plot = results shown as map and as table

### Export data

In [ ]:
area.to_csv(os.path.join(out_folder, "pop_flooded_rp100.csv"))

In [ ]:
out_folder = os.path.join(data_dir, "output")
if not os.path.exists(out_folder):
    os.mkdir(out_folder)